In [1]:
import pandas as pd
df = pd.read_csv("Reviews.csv")
print(df)

            Id   ProductId          UserId                      ProfileName  \
0            1  B001E4KFG0  A3SGXH7AUHU8GW                       delmartian   
1            2  B00813GRG4  A1D87F6ZCVE5NK                           dll pa   
2            3  B000LQOCH0   ABXLMWJIXXAIN  Natalia Corres "Natalia Corres"   
3            4  B000UA0QIQ  A395BORC6FGVXV                             Karl   
4            5  B006K2ZZ7K  A1UQRSCLF8GW1T    Michael D. Bigham "M. Wassir"   
...        ...         ...             ...                              ...   
568449  568450  B001EO7N10  A28KG5XORO54AY                 Lettie D. Carter   
568450  568451  B003S1WTCU  A3I8AFVPEE8KI5                        R. Sawyer   
568451  568452  B004I613EE  A121AA1GQV751Z                    pksd "pk_007"   
568452  568453  B004I613EE   A3IBEVCTXKNOH          Kathy A. Welch "katwel"   
568453  568454  B001LR2CU2  A3LGQPJCZVL9UC                         srfell17   

        HelpfulnessNumerator  HelpfulnessDenominato

In [2]:
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [3]:
import nltk
import re
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rajashree/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]') #replace occurrences of these by space
STOPWORDS = set(stopwords.words('english'))
DIGITS = r'[0-9]'
BAD_SYMBOLS_RE = r'[#+_"]' #remove occurrences of these
PUNCTUATION = r'[^\w\s]'

def clean_text(text):
  text = text.lower() # lowercase text
  text = re.sub(PUNCTUATION, '', text)
  text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
  text = re.sub(BAD_SYMBOLS_RE, '', text )
  text = re.sub(DIGITS, '', text)
  text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
  return text

In [5]:
# clean text
df['Text'] = df['Text'].apply(clean_text)
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,bought several vitality canned dog food produc...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,product arrived labeled jumbo salted peanutsth...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",confection around centuries light pillowy citr...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,looking secret ingredient robitussin believe f...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,great taffy great price wide assortment yummy ...


In [8]:
df = df.drop(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator', 'HelpfulnessDenominator', 'Score', 'Time'], axis=1)

In [9]:
df.head()

,Summary,Text
0,Good Quality Dog Food,bought several vitality canned dog food produc...
1,Not as Advertised,product arrived labeled jumbo salted peanutsth...
2,"""Delight"" says it all",confection around centuries light pillowy citr...
3,Cough Medicine,looking secret ingredient robitussin believe f...
4,Great taffy,great taffy great price wide assortment yummy ...


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_count = TfidfVectorizer(ngram_range = (1,2))
final_counts_tfidf = tfidf_count.fit_transform(df['Text'].values)
print(final_counts_tfidf.shape)
print(type(final_counts_tfidf))

(568454, 4795035)
<class 'scipy.sparse.csr.csr_matrix'>


In [12]:
# To get feature names
features = tfidf_count.get_feature_names_out()
print(len(features))

4795035


In [13]:
print(features[100000:100010])

['alone thing' 'alone things' 'alone think' 'alone thinking'
 'alone thinks' 'alone though' 'alone thought' 'alone three'
 'alone throughout' 'alone throwing']


In [14]:
# Train your own Word2Vec model using your own text corpus
import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
list_of_sent=[]
for sent in df['Text'].values:
    list_of_sent.append(sent.split())
print(df['Text'].values[0])
print("*****************************************************************")
print(list_of_sent[0])

bought several vitality canned dog food products found good quality product looks like stew processed meat smells better labrador finicky appreciates product better
*****************************************************************
['bought', 'several', 'vitality', 'canned', 'dog', 'food', 'products', 'found', 'good', 'quality', 'product', 'looks', 'like', 'stew', 'processed', 'meat', 'smells', 'better', 'labrador', 'finicky', 'appreciates', 'product', 'better']


In [16]:
# min_count = 5 considers only words that occured atleast 5 times
W2V_model = Word2Vec(list_of_sent, min_count = 5, workers = 4)
w2v_words = list(W2V_model.wv.vocab)
print("Number of words that occured minimum 5 times", len(w2v_words))
print("Sample words", w2v_words[0:25])

AttributeError: The vocab attribute was removed from KeyedVector in Gensim 4.0.0.
Use KeyedVector's .key_to_index dict, .index_to_key list, and methods .get_vecattr(key, attr) and .set_vecattr(key, attr, new_val) instead.
See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4